# ADK Application Testing

This notebook demonstrates how to test an ADK (Agent Development Kit) application.
It covers both local and remote testing, both with Agent Engine and Cloud Run.

<img src="https://github.com/GoogleCloudPlatform/agent-starter-pack/blob/main/docs/images/adk_logo.png?raw=true" width="400">


## Install dependencies

In [ ]:
!pip install google-cloud-aiplatform --upgrade

### Import libraries

In [1]:
import json
from pathlib import Path
import requests

## If you are using Agent Engine
See more documentation at [Agent Engine Overview](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview)

### Remote Testing

In [4]:
# Replace with your Agent Engine ID
AGENT_ENGINE_ID = "projects/PROJECT_ID/locations/us-central1/reasoningEngines/ENGINE_ID"

In [5]:
remote_agent_engine = vertexai.agent_engines.get(AGENT_ENGINE_ID)

In [ ]:
for event in remote_agent_engine.stream_query(message="hi!", user_id="test"):
    print(event)

In [ ]:
remote_agent_engine.register_feedback(
    feedback={
        "score": 5,
        "text": "Great response!",
        "invocation_id": "test-invocation-123",
        "user_id": "test",
    }
)

### Local Testing

You can import directly the AgentEngineApp class within your environment. 
To run the agent locally, follow these steps:
1. Make sure all required packages are installed in your environment
2. The recommended approach is to use the same virtual environment created by the 'uv' tool
3. You can set up this environment by running 'make install' from your agent's root directory
4. Then select this kernel (.venv folder in your project) in your Jupyter notebook to ensure all dependencies are available

In [ ]:
from app.agent import root_agent
from app.agent_engine_app import AgentEngineApp

agent_engine = AgentEngineApp(agent=root_agent)

In [ ]:
agent_response = agent_engine.query(message="hi!", user_id="test")

### Local Testing

> You can run the application locally via the `make backend` command.

#### Create a session
 Create a new session with user preferences and state information


In [35]:
with open("../tests/samples/diff.txt", "r") as f:
    git_diff = f.read()

user_id = "test_user_123"
session_id = "test_session_456"
session_data = {"state": {"preferred_language": "English", "visit_count": 1}, "git_diff": git_diff}

session_url = f"http://127.0.0.1:8501/apps/app/users/{user_id}/sessions/{session_id}"
headers = {"Content-Type": "application/json"}

session_response = requests.post(session_url, headers=headers, json=session_data)
print(f"Session creation status code: {session_response.text}")

Session creation status code: {"id":"test_session_456","app_name":"app","user_id":"test_user_123","state":{"state":{"preferred_language":"English","visit_count":1},"git_diff":"diff --git a/main.py b/main.py\nindex 062670c..eac224e 100644\n--- a/main.py\n+++ b/main.py\n@@ -1,7 +1,42 @@\n from fastapi import FastAPI\n+from contextlib import asynccontextmanager\n+import aiosqlite\n\n-app = FastAPI()\n+@asynccontextmanager\n+async def lifespan(app: FastAPI):\n+    # Create and connect to the database\n+    db = await aiosqlite.connect('users.db')\n+    cursor = await db.cursor()\n+    await cursor.execute('''\n+        CREATE TABLE IF NOT EXISTS users (\n+            username TEXT PRIMARY KEY,\n+            email TEXT\n+        )\n+    ''')\n+    await cursor.execute(\"INSERT OR IGNORE INTO users (username, email) VALUES ('alice', 'alice@example.com')\")\n+    await cursor.execute(\"INSERT OR IGNORE INTO users (username, email) VALUES ('bob', 'bob@example.com')\")\n+    await db.commit()\n

#### Send a message
Send a message to the backend service and receive a streaming response


In [36]:
message_data = {
    "app_name": "app",
    "user_id": user_id,
    "session_id": session_id,
    "new_message": {"role": "user", "parts": [{"text": "Review PR code changes according to your instructions."}]},
    "streaming": False,
}

message_url = "http://127.0.0.1:8501/run"
message_response = requests.post(
    message_url, headers=headers, json=message_data, stream=False
)

print(f"Message send status code: {message_response.status_code}")
print(message_response.text)

Message send status code: 200
[{"content":{"parts":[{"functionCall":{"id":"adk-ed6b0b2d-0e1d-4f50-b727-ec8b02a8f7bb","args":{"agent_name":"VulnerabilityFixerAgent"},"name":"transfer_to_agent"}}],"role":"model"},"invocation_id":"e-98337052-00b8-4531-8f83-d026ebcced7c","author":"root_agent","actions":{"state_delta":{},"artifact_delta":{},"requested_auth_configs":{}},"long_running_tool_ids":[],"id":"hBNV8r5a","timestamp":1749412591.832031},{"content":{"parts":[{"functionResponse":{"id":"adk-ed6b0b2d-0e1d-4f50-b727-ec8b02a8f7bb","name":"transfer_to_agent","response":{}}}],"role":"user"},"invocation_id":"e-98337052-00b8-4531-8f83-d026ebcced7c","author":"root_agent","actions":{"state_delta":{},"artifact_delta":{},"transfer_to_agent":"VulnerabilityFixerAgent","requested_auth_configs":{}},"id":"C7u0AcPO","timestamp":1749412595.944732},{"content":{"parts":[{"text":"{\n  \"patches\": [\n    {\n      \"file\": \"main.py\",\n      \"position\": 24,\n      \"patch\": \"```suggestion\\n    db = awai